# 04 - Kriging
Ejecuta OK sobre la grilla.

In [ ]:
import os, sys, json

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir(PROJECT_ROOT)
else:
    PROJECT_ROOT = os.getcwd()

sys.path.insert(0, PROJECT_ROOT)

In [ ]:
from src.io_utils import read_csv_robust, standardize_columns, coerce_numeric
from src.grid import make_grid_dataframe, grid_from_extents
from src.variography import experimental_variogram_2d, fit_variogram_model
from src.kriging import ordinary_kriging_2d
from src.eda import plot_xy_scatter

with open("config/project.json", "r", encoding="utf-8") as f:
    cfg = json.load(f)

if not os.path.exists(cfg["data_csv_path"]):
    from src.make_demo_data import make_demo_csv
    make_demo_csv(cfg["data_csv_path"])

df = read_csv_robust(cfg["data_csv_path"])

mapping = {
    "x": cfg["columns"].get("x"),
    "y": cfg["columns"].get("y"),
    "z": cfg["columns"].get("z"),
    "var": cfg["columns"].get("variable_objetivo"),
}

df = standardize_columns(df, mapping)
coerce_numeric(df, ["x", "y", "z", "var"])
df.dropna(subset=["x", "y", "var"], inplace=True)

# Grid
grid_cfg = cfg.get("grid", {})
if grid_cfg.get("auto_from_data", True):
    grid_spec = grid_from_extents(
        df,
        "x",
        "y",
        "z",
        float(grid_cfg.get("dx", 25.0)),
        float(grid_cfg.get("dy", 25.0)),
        float(grid_cfg.get("dz", 5.0)),
        pad=float(grid_cfg.get("pad", 0.0)),
    )
else:
    grid_spec = {
        "nx": grid_cfg["nx"],
        "ny": grid_cfg["ny"],
        "nz": grid_cfg.get("nz", 1),
        "xmin": grid_cfg["xmin"],
        "ymin": grid_cfg["ymin"],
        "zmin": grid_cfg.get("zmin", 0.0),
        "dx": grid_cfg["dx"],
        "dy": grid_cfg["dy"],
        "dz": grid_cfg.get("dz", 1.0),
    }

grid_df = make_grid_dataframe(grid_spec)

# Variogram model
exp = experimental_variogram_2d(df, "x", "y", "var", cfg["variogram"])
model = fit_variogram_model(exp, float(df["var"].var(ddof=1)), model_type="spherical")

kriged = ordinary_kriging_2d(df, "x", "y", "var", grid_df, model, cfg["kriging"])

os.makedirs("outputs/figures", exist_ok=True)
os.makedirs("outputs/tables", exist_ok=True)
plot_xy_scatter(kriged, "x", "y", "estimate", "outputs/figures/kriging_estimate.png")
kriged.to_csv("outputs/tables/kriging_var.csv", index=False)